In [0]:
import pandas as pd
import numpy as np
np.random.seed(4869)

In [107]:
# CODE SNIPPET TO ACCESS THE FILES IN GOOGLE DRIVE (GO TO BROWSER AND VERIFY)
# THEN YOU CAN ACCESS THE FILES ON LEFT SIDEBAR (copy path)
# (f)

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
# drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
# !ls "/content/drive/My Drive"

Mounted at /content/drive


In [108]:
listing_l = pd.read_csv('drive/My Drive/roofstock data/roofstock_marketplace_listing_historical_full.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11,14,38,42,43,57,58,59,64,65,66,80,81,84,132,141,151,166,167,168,169,170) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
for i in range(1,len(listing_l)):
    if (listing_l.loc[i,'LISTING_STATUS'] == 'For Sale') or (listing_l.loc[i,'LISTING_STATUS'] == 'Coming Soon'):
        if pd.notnull(listing_l.loc[i-1, 'LISTING_PUBLISH_TS']):
            listing_l.loc[i,'LISTING_PUBLISH_TS'] = listing_l.loc[i-1, 'LISTING_PUBLISH_TS']
        else:
            listing_l.loc[i,'LISTING_PUBLISH_TS'] = listing_l.loc[i,'REC_START_TS']
    else:
        listing_l.loc[i,'LISTING_PUBLISH_TS'] = np.nan

In [111]:
listing_l.shape

(288760, 172)

In [0]:
listing = listing_l.drop(columns=['PROPERTYSUMMARY','PROPERTYHIGHLIGHTS','PROPERTYMARKETINGPOINTS'])

In [0]:
trans = pd.read_csv('drive/My Drive/roofstock data/roofstock_transaction_history.csv')
offers = pd.read_csv('drive/My Drive/roofstock data/roofstock_marketplace_offers_full.csv')

In [0]:
for_sale = listing[listing['LISTING_STATUS']=='For Sale']
soon = listing[listing['LISTING_STATUS']=='Coming Soon']
for_sale = pd.concat([for_sale,soon]) 
for_sale = for_sale.sort_values(by=['LISTING_ID','LISTING_PUBLISH_TS'])
for_sale = listing[listing['LIST_PRICE'].notnull()]

In [127]:
for_sale.shape

(214410, 169)

In [0]:
for_sale = for_sale.groupby(['LISTING_ID','LISTING_PUBLISH_TS']).last().reset_index()

In [0]:
for_sale = for_sale[for_sale['LIST_PRICE'].notnull()]
for_sale['START_DATE'] = pd.to_datetime(for_sale['LISTING_PUBLISH_TS'])

In [130]:
for_sale.shape

(6094, 170)

In [0]:
### OFFERS & TRANS

In [132]:
offers = offers[['OFFER_ITERATION_SUBMIT_DATE','LISTING_ID','OFFER_PRICE','OFFER_DIRECTION','OFFER_STATUS']]
offers['START_DATE'] = np.nan
offers['LIST_PRICE'] = 0
offers.shape

(31852, 7)

In [0]:
offers['OFFER_ITERATION_SUBMIT_DATE'] = pd.to_datetime(offers['OFFER_ITERATION_SUBMIT_DATE'])
#.apply(lambda x: x.tz_localize('UTC')) 

In [135]:
for i in range(0,len(offers)):
    df = for_sale[for_sale['LISTING_ID']==offers.loc[i,'LISTING_ID']]
    df = df.reset_index()
    for j in range(0,len(df)):
        if df.loc[j,'START_DATE'] <= offers.loc[i,'OFFER_ITERATION_SUBMIT_DATE']:
            offers.loc[i,'LIST_PRICE'] = df.loc[j,'LIST_PRICE']
            offers.loc[i,'START_DATE'] = df.loc[j,'START_DATE']
            break
        else:
            pass
offers.shape

(31852, 7)

In [136]:
offers['diff'] = (offers['OFFER_PRICE'] - offers['LIST_PRICE'])/offers['LIST_PRICE'] *100
good_offer = offers[offers['diff']>=-10]
good_offer_1 = good_offer[good_offer['OFFER_DIRECTION']=='BuyerToSeller']
good_offer_2 = good_offer[(good_offer['OFFER_DIRECTION']=='SellerToBuyer') & (good_offer['OFFER_STATUS'] == 'Accepted')]
good_offer = pd.concat([good_offer_1,good_offer_2]) 
good_offer.shape

(14942, 8)

In [137]:
good_offer.head()

,OFFER_ITERATION_SUBMIT_DATE,LISTING_ID,OFFER_PRICE,OFFER_DIRECTION,OFFER_STATUS,START_DATE,LIST_PRICE,diff
0,2019-06-05 00:00:00+00:00,1720117,85000,BuyerToSeller,Countered,2019-05-11 23:30:16+00:00,89500.0,-5.027933
4,2017-07-05 00:00:00+00:00,1627157,80000,BuyerToSeller,New,NaN,0.0,inf
9,2018-04-06 00:00:00+00:00,1658925,110000,BuyerToSeller,New,NaN,0.0,inf
10,2018-04-13 00:00:00+00:00,1658979,95000,BuyerToSeller,Countered,NaN,0.0,inf
12,2018-01-16 00:00:00+00:00,1636213,61000,BuyerToSeller,Cancelled,2017-10-26 06:02:36+00:00,66389.0,-8.117309


In [138]:
# Transaction
trans = trans[trans['OFFER_ID']==-99]
trans = trans[['TRANSACTION_CREATED_DATE','LISTING_ID','SALE_PRICE']]
trans['START_DATE'] = np.nan
trans['LIST_PRICE'] = 0
trans['TRANSACTION_CREATED_DATE'] = pd.to_datetime(trans['TRANSACTION_CREATED_DATE'])
trans = trans.reset_index()
trans.shape

(2258, 6)

In [0]:
trans['TRANSACTION_CREATED_DATE'] = pd.to_datetime(trans['TRANSACTION_CREATED_DATE']).apply(lambda x: x.tz_localize('UTC')) 

In [140]:
good_trans = trans[['TRANSACTION_CREATED_DATE','LISTING_ID','SALE_PRICE','START_DATE','LIST_PRICE']]
good_trans.shape

(2258, 5)

In [141]:
main_offer = good_offer[['OFFER_ITERATION_SUBMIT_DATE','LISTING_ID','LIST_PRICE','OFFER_PRICE','START_DATE']]
main_offer = main_offer.rename(columns={'OFFER_ITERATION_SUBMIT_DATE':'END_DATE'})
main_offer['END_DATE'] = pd.to_datetime(main_offer['END_DATE'])
main_offer.shape

(14942, 5)

In [142]:
main_trans = good_trans[['TRANSACTION_CREATED_DATE','LISTING_ID','LIST_PRICE','SALE_PRICE','START_DATE']]
main_trans = main_trans.rename(columns={'TRANSACTION_CREATED_DATE':'END_DATE',
                                       'SALE_PRICE':'OFFER_PRICE'})
main_trans['END_DATE'] = pd.to_datetime(main_trans['END_DATE'])
main_trans.shape

(2258, 5)

In [155]:
main = pd.concat([main_trans,main_offer])
main.shape

(17200, 5)

In [156]:
main = main[main['LIST_PRICE'].notnull()]
main.shape

(17200, 5)

In [157]:
sum(main['START_DATE'].isnull())

10460

In [158]:
sum(main['START_DATE'].notnull())

6740

In [163]:
main = main[main['START_DATE'].notnull()]
main['START_DATE'] = pd.to_datetime(main['START_DATE']).apply(lambda x: x.date())
main['START_DATE'] = main['START_DATE'].astype(str)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
for_sale['START_DATE'] = pd.to_datetime(for_sale['START_DATE']).apply(lambda x: x.date())
for_sale['START_DATE'] = for_sale['START_DATE'].astype(str)

In [0]:
main = main.merge(for_sale, on = ['LISTING_ID','START_DATE'])

In [169]:
main.shape

(6756, 173)

In [0]:
main.to_csv('drive/My Drive/roofstock data/ultimate_main.csv',index=False)